# Zonas Seguras y registro de Caminos

## Contexto y Teoria

Este proyecto es un complemento propio que busca de forma alterna mejorar el analisis y la visualización de los sistemas de deteccion.

Un sistema de deteccion hace referencia a un sistema de vigilancia con camara, el cual tiene la capacidad de reconocer objetos que aparecen el la imagen detectada, en la mayoria de los casos humanos.
Cuando el sistema detecta un objeto de interes captura la imagen y lo guarda, en algunos casos es posible no solo guardar la lista de imagenes capturadas, sino la lista de coordenadas que registran a los objetos de interes, este proyecto busca trabajar con dichas coordenadas y facilitar el analizis de grandes grupos de datos para generar un mapa que permita conocer los caminos donde la concurrencia de los objetos de interes es mayor, o detectar lugares donde esa concurrencia es nula.




En este proyecto vamos a referirnos a la imagen objetivo como el mapa de calor del lugar, esto como referencia a que buscamos los caminos donde podriamos encontrar objetos de interes, donde de forma divertida le podemos decir caliente o frio segun que tan lejos o cerca estamos de ese objeto.

Para realizar esta imagen se realizara un objeto de pintado, donde a una imagen en negro se le pondra una capa de blanco encima en cada sitio que estuviera un objeto de interes, como dejando un rastro, con lo que al final todas las coordenadas nos debe quedar una imagen con diversos blancos de intensidad segun cuanto pintamos en cada lugar.

las imagenes tienen una ubicacion donde la esquina superior izquierda es 0,0 y la inferior derecha es el maximo tanto en X como en Y.

Cabe resaltar que los objetos de interes estan representados como cuadrados con dos coordenadas, esquina superior izquierda como inicio de figura, y esquina inferior derecha como final del cuadrado, y se representan como cuadrados debido que es la forma que el programa de deteccion de objetos lo almacena, asi que por facilidad se realizo compatibilidad con ese proyecto.

## Objetivos

1. Vamos a realizar un programa que con dada una lista de coordenadas que representan un objeto de interes genere una imagen que represente la frecuencia en cada posicion de dicho objeto.

2. Tambien se busca que esta imagen se pueda guardar de forma comoda para el uso en otras aplicaciones.
3. Es deseable que la imagen se separe tambien  de forma que se pueda visualizr de forma separada las zonas con mayor o menos frecuencia.
4. Crear un metodo para simular las coordenadas y por lo cual tener ejemplos previsibles que probar.

## Codigo

### Instalar e importar las librerias usadas

Se descarga la libreria tifffile para tener la capacidad de leer y generar archivos en .tiff
Se utiliza esta terminacion debido a dos razones, la primera es que tiff guarda valores precisos, lo que nos permite tener valores mas complejos y precisos al realizar el proceso, si fueran imagenes normales como png las unidades serian int8, que son enteros de 0 a 255, en tiff podemos usar el mismo rango si lo deseamos pero trabajar con valores flotantes de 32 o 64 bites, en nuestro caso decidimos trabajar con valores flotantes de 32 bits.

La segunda razon es que tiff puede guardar paginas, lo que nos permite guardar en un mismo archivo el mapa de calor, y sus diversas capas deseadas,
siendo una capa los lugares donde se pasa con entre X frecuencias para facilitar el analisis y busqueda.

In [14]:
%pip install tifffile

Note: you may need to restart the kernel to use updated packages.


Las librerias que usaremos en totalidad seran numpy, tifffile y random.
Previamente se explico la razon de tiffile, random sera usado para la generacion de coordenadas del 4 objetivo.
En python la mejor forma para generar analizar y editar arreglos es con la libreria numpy, tanto asi que pasar de tifffile  a numpy y al contrario se realiza de forma natural (al igual que con otros lectores de imagenes).

In [15]:
import numpy as np
import tifffile
import random


### Procesamiento de archivos



Lo primero que necesitamos para analizar las coordenadas son los datos, para importar o crear la imagen se creo la funcion procesar_entrada().

procesar_entrada recibe el nombre con extension de la imagen de referencia, ej. 'mapaDeCalor.tiff' (cuando se encuentra en la misma carpeta del archivo), o el tamaño a analizar de la imagen definidos en n y m.

el codigo se divide en dos partes, la primera que accede si la direccion de imagen 'imagenDir' existe, con lo cual utilizando la libreria tifffle lee el archivo en esa direccion, guarda la primera pagina del archivo en 'heatValue', que representa el mapa de calor previo o existente, si existen mas paginas las guarda como extras, por ultimo guarda en 'size' el tamaño de 'heatValue' y devuelve ambos, para usarse en otros lados.

en caso de no existir direccion de archivo, procesar_entrada toma n y m y genera un numpy de zeros de ese tamaño para 'heatValue', este numpy es una matriz que representa la imagen, en cualquiera de los dos casos tomados, tambien se a de mencionar que la razon por la que es un arreglo de zeros es debido a que 0 equivale a negro en color, como mencionamos arriba queremos un lienzo en negro.

In [16]:
def procesar_entrada(imagenDir=None, n=None, m=None):

    if imagenDir is not None:
        with tifffile.TiffFile(imagenDir) as tiff:
            num_paginas = len(tiff.pages)
    
            if num_paginas > 1:
                print("El archivo TIFF es multipágina.")
                
                # Leer la primera página
                heatValue = tiff.pages[0].asarray()
                size = heatValue.shape
                print("Primera página leída con tamaño:", heatValue.shape)
                
                # Leer las páginas restantes
                extrasCapas = [tiff.pages[i].asarray() for i in range(1, num_paginas)]
                print(f"Leídas {len(extrasCapas)} páginas adicionales.")
            else:
                print("El archivo TIFF es de una sola página.")
                # Leer la imagen
                heatValue = tiff.pages[0].asarray()
                size = heatValue.shape
                print("Imagen leída con tamaño:", heatValue.shape)
    elif n is not None and m is not None:
        size =[n,m]
        # Crear una imagen vacía (negra) de tamaño n x m
        heatValue = np.zeros((n, m),  dtype=np.float32)
        print(f"Imagen de tamaño {n} x {m} creada.")
    
    else:
        raise ValueError("Se debe proporcionar una imagen o los valores n y m.")
    return heatValue, size

GuardarArchivo es una funcion que realiza lo opuesto a procesar_entrada, recibe el nombre del archivo destino en 'nombreDeArchivo',el mapa de calor en 'heatValue', y si se guardaran las paginas extras donde mostraremos los diversos mapas secundarios se puede dar 'multiple' que al ser verdadero realiza ese proceso junto con 'extras' que contiene esas paginas extra.

In [17]:
def guardarArchivo(nombreDeArchivo,heatValue, direccion =None,multiple =False,extras = None):
    with tifffile.TiffWriter(nombreDeArchivo+".tiff") as tiff:
        tiff.write(heatValue)
        if(multiple and extras != None):
            for i in range(len(extras)):
                tiff.write(extras[i])



#### Generacion de coordenadas



Dentro de los archivos existe uno que va con los objetivos de forma indirecta, en el siguiente bloque definimos como se van a simular una serie de coordenadas y guardarlas.

En estas funciones hay tres variables recurrentes en significado

r = cantidad de coordenadas deseadas.
n = cantidad de filas de la imagen
m = cantidad de columnss de la imagen

procesoRandom(), 
tomando esos tres valores da genera todas las combinaciones posibles de coordenadas XY1,XY2, donde XY1 sea menor a XY2, luego usando la libreria random nos da un muestra de tamaño r de esas coordenadas, que al incluir todas nos da una muestra aleatoria de coordenadas.

cubePlay(), busca simular mas el camino de un objeto de interes en la imagen.
creando un punto centrado en alguno de los bordes de la imagen y un tamaño aleatorio relativo al tamaño de la imagen, emula r veces el proceso del cubo desde su punto inicial moviendose dentro de la imagen, lo que suele dar dos puntos de alta frecuencia o incluso uin camino visible.



In [18]:
def procesoRandom(r,n,m):
    valoresX = list(range(0, m))
    valoresY = list(range(0, n))

    parX = [(a, b) for i, a in enumerate(valoresX) for b in valoresX[i+1:]]
    parY = [(a, b) for i, a in enumerate(valoresY) for b in valoresY[i+1:]]
    
    # Seleccionar n pares aleatorios
    duplasX = random.sample(parX, n)
    duplasY = random.sample(parY, n)

    cords = [[[d1[0],d2[0]],[d1[1],d2[1]]] for d1, d2 in zip(duplasX, duplasY)]

    duplas = random.sample(cords, r)
    
    return duplas

def cubePlay(r,n,m):
    center =[0,0]
    exi = random.randint(0,3)
    if(exi ==0):
        center = [0,random.randint(0,n)]
    elif(exi ==1):
        center = [m,random.randint(0,n)]
    elif(exi ==2):
        center = [random.randint(0,m),0]
    else:
        center = [random.randint(0,m),n]

    
    mel = min(n,m)
    rad = random.randint(mel/8,mel/4)

    
    duplas = []

    for i in range(r):

        
        x1 =center[0]-rad
        x2 =center[0]+rad

        x1 = 0 if x1<0 else x1

        x2 = m if x2>m else x2

        y1 =center[1]-rad
        y2 =center[1]+rad

        y1 = 0 if y1<0 else y1

        y2 = n if y2>n else y2

        duplas.append([[x1,y1],[x2,y2]])

        move = [random.randint(-mel*0.2,mel*0.2),random.randint(-mel*0.2,mel*0.2)]

        move[0] = (-1 if (center[0]+move[0]<0 or center[0]+move[0]>m) else 1) * move[0]
        move[1] = (-1 if (center[1]+move[1]<0 or center[1]+move[1]>n) else 1) * move[1]

        center =[center[0]+move[0],center[1]+move[1]]
        

    return duplas
    


generarProces(), esta funcion genera el muestreo con una variable randomm, utilizando esta variable para decidir entre las dos funciones previas para generar las coordenadas.

GuardarCords(), recibe la lista de coordenadas y el nombre del archivo, con lo que procede a guardar cada coordenada en un txt para leer luego.

LeerCoordenadas(), recibe el nombre del archivo sin .txt, busca el archivo y retorna la lista de coordenadas.

In [ ]:


def generarProces(r,n,m,randomm = True):
    if(randomm):
        return procesoRandom(r,n,m)
    else:
        return cubePlay(r,n,m)
    

def GuardarCords(cordss,name):
    with open(name+'.txt', 'w') as f:
        for coord in cordss:
            f.write(f"{coord}\n")

    print("Coordenadas guardadas en '"+name+"'.txt'.")

def LeerCoordenadas(name):
    coordenadas_leidas = []
    with open(name+'.txt', 'r') as f:
        for line in f:
            coordenadas_leidas.append(eval(line.strip()))

    return coordenadas_leidas 

aqui mostramos el proceso mas simple para guardar 100 coordenadas en una iumagen 1280*720, utilizando la funcion cubePlay para randomizar.

In [19]:
name = "coordenadas"
GuardarCords(generarProces(100,1280,720,False),name)

Coordenadas guardadas en 'coordenadas'.txt'.


### Mapa de Calor

!(advertencia)
Debido al funcionamiento de python las funciones se encuentran en orden de profundidad (primero aquellas que son llamadas por otras) por lo que las que se mostraran siguientes no tienen mucho contexto de su uso hasta llegar mas adelante.

#### Manejo de matrices y desviaciones



Para  manipular la informacion dentro de las matrices se generaron diversas funciones con diferentes objetivos:

normalizar_matriz(), la mas directa funcion, recibe una matriz numpy, y la busca convertir a un rango 0 a 1 segun su minimo y maximo respectivamente.
Por fuerza para nuestro caso el minimo de la matriz no normalizada siempre sera 0 incluso si se cubiera toda la imagen alguna vez para la correcta relacion, por lo que mientras el maximo de la matriz sea diferente de 0 podemos dividir cada valor de la matriz por el maximo, que con numpy se realiza como si la matriz fuera un valor unico.

A la hora de comparar dos imagenes de Calor se buscaron dos metodos diferentes, uno donde se realiza por pesos, y otro por experiencia.

CalcularNuevaDesviacionAlfa(), calcula los valores nuevos al combinar una imagen de calor vieja y una nueva, este proceso depende de un 'alpha', que es el peso de la imagen vieja, siendo el nuevo valor del mapa la suma de pesos de el mapa viejo y el nuevo, el peso del mapa nuevo es 1-'alpha', lo que nos da una aproximacion bastante consistente y nos permite considerar que tan  importante queremos que sea el nuevo mapa en el aprendizaje.

CalculaNuevaDesviacion(), calcula los valores nuevos al combinar una imagen de calor vieja y una nueva, busca trabajar con un sistema de pesos individual para cada posicion, en la cual ese peso es el mapa viejo (tomandolo como que tan importante es ese lugar para modificarlo),  a este peso le multiplicamos con la diferencia del nuevo y el viejo mapa, que nos da que tan diferentes son los datos entre si, junto con la direccion a la que el nuevo mapa desea que se cambie el viejo.


In [20]:
def normalizar_matriz(matriz):
    #convierte la matriz a un rango de 0 a 1
    max_val = np.max(matriz)
    if max_val == 0:
        return matriz  # Evitar división por cero si todos los valores son iguales
    return (matriz) / (max_val)

def CalcularNuevaDesviacionAlfa(preHeat, newHeat,alpha): ## formula para calcular el efecto del video sobre los datos previos, utilizando un valor alfa de peso
    # Calcular con el facvotr de suavisado
    newnewHeat = alpha*preHeat + (1-alpha) * newHeat
    return newnewHeat

def CalcularNuevaDesviacion(preHeat, newHeat): ## formula para calcular el efecto del video sobre los datos previos,
    
    diferencia = preHeat - newHeat
    
    producto = preHeat * diferencia

    # Calcular el nuevo valor
    newnewHeat = preHeat + producto

    return newnewHeat



#### Pintar las coordenadas y agregarlas al lienzo antiguo

procesarCamino(), en este proceso juntamos todo lo que hemos almacenado para crear el mapa de calor, 

creamos un lienzo en negro y lo pintamos en uno cada vez que en esa posicion se detecto un objeto de interes.
luego normalizamos tanto el nuevo mapa como el viejo, 
dependiendo si se quiere usar o no alfa usamos una funcion de desviacion diferente.
y por ultimo retornamos elnuevo mapa normalizado.

In [21]:
def procesarCamino(heatValue,size,newPath,calor=False, alpha = 1):
    
    newHeat = np.zeros(heatValue.shape,  dtype=np.float32)
    
    for (x1, y1), (x2, y2) in newPath:
        newHeat[x1:x2+1, y1:y2+1] += 1
    # Normalizar la matriz a un rango de 0 a 255
    newHeat = normalizar_matriz(newHeat)
    heatValue = normalizar_matriz(heatValue)
    if(calor):
        newHeat = CalcularNuevaDesviacionAlfa(heatValue,newHeat,alpha)
    else:
        newHeat = CalcularNuevaDesviacion(heatValue,newHeat)
        
    return newHeat

## Muestra por partes

dividirPorFases(), esta funcion toma un mapa y un valor de division, ya sea en cuanto porcentaje vale cada division, o cuantas divisiones desea.

con esto crea una lista de copias en negro de el mapa, y recorriendolo por completo pinta en cada copia donde los valores entran dentro de su particion.

estos valores quedan en unit8 debido a que la imagn solo guarda blanco y negro si esta o no en el rango.

In [22]:
def dividirPorFases(mapa, division, porcent=True):
    percent = 1
    raz = 1
    filas,columnas = mapa.shape

    if(not porcent):
        raz = int(division)
        percent = 1/division 
    else:
        raz = int(1/division)
        percent = division 
    extra = []

    for k in range(raz):
        print(percent*k*255)
        copy  = np.zeros(mapa.shape,dtype=np.uint8)
        for i in range(filas):
            for j in range(columnas):
                if(mapa[i,j]>=percent*k and mapa[i,j]<=percent*(k+1)):
                    copy[i,j] =255
        extra.append(copy)

    return extra





procesamientoCaminos(), este metodo almacena todos los procesos previos para poder realizar un mapa dandole solo el nombre del archivo, para que busque las coordenadas en el archivo coordenadas.

procesamientoCaminosNN(), este archivo realiza el mismo proceso pero no toma una imagen previa, por lo que el proceso es totalmente nuevo.

In [23]:
def procesamientoCaminos(name,n=720,m=1280,calor=True,alpha=0.75,divisions=5):
    heatValue,size = procesar_entrada(name+".tiff", n=n,m=m)
    newPath = LeerCoordenadas("coordenadas")
    # Dimensiones de la imagen (n: alto, m: ancho
    newheatValue = procesarCamino(heatValue,size,newPath,calor,alpha)
    extrasCapas = dividirPorFases(newheatValue,division=5,porcent=False)

    guardarArchivo(name,newheatValue,multiple=True,extras=extrasCapas)

def procesamientoCaminosNN(name,n=720,m=1280,calor=True,alpha=0.0,divisions=5):
    heatValue,size = procesar_entrada(n=n,m=m)
    newPath = LeerCoordenadas("coordenadas")
    # Dimensiones de la imagen (n: alto, m: ancho
    newheatValue = procesarCamino(heatValue,size,newPath,calor,alpha)
    extrasCapas = dividirPorFases(newheatValue,division=5,porcent=False)

    guardarArchivo(name,newheatValue,multiple=True,extras=extrasCapas)

Aqui tenemos un ejemplo de como se llama a un nuevo mapa de calor, generando nuevas coordenadas para este.

In [26]:
name = "coordenadas"
GuardarCords(generarProces(100,1280,720,False),name)
procesamientoCaminosNN('mapaDeCalor',calor=False)



Coordenadas guardadas en 'coordenadas'.txt'.
mapaDeCalor.tiff
El archivo TIFF es multipágina.
Primera página leída con tamaño: (720, 1280)
Leídas 5 páginas adicionales.
5.0
0.3125
0.0


C:\Users\User\AppData\Local\Temp\ipykernel_21344\2566342434.py:14: RuntimeWarning: invalid value encountered in divide
  direccion = np.where(magnitud_diferencia == 0, 0, diferencia / magnitud_diferencia)  # saber la direccion en la que nos desviamos si el valor es el mismo no debe haber cambio por ende es 0


51.0
102.0
153.00000000000003
204.0
